# 

# Detecção de conflitos em projetos BIM usando grafos e ontologias

## Introdução
Este projeto tem como objetivo detectar conflitos em projetos BIM (Building Information Modeling) utilizando grafos e ontologias. A detecção de conflitos é uma etapa crucial no processo de planejamento e execução de projetos de construção, pois ajuda a identificar problemas potenciais em projetos antes que eles se tornem caros ou difíceis de resolver.

## Tecnologias Utilizadas
- **Python**: Linguagem de programação principal para o desenvolvimento do projeto.
- **PyShacl**: Biblioteca para manipulação de grafos.
- **RDFLib**: Biblioteca para trabalhar com grafos RDF (Resource Description Framework).
- **Ontologia**: Definição de classes, propriedades e relações entre os elementos do projeto BIM.
- **Neo4j**: Banco de dados orientado a grafos para armazenar e consultar os dados do projeto.

### Importando as bibliotecas necessárias

In [1]:
import ifcopenshell
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL
import re
from rdflib.namespace import SH
from pyshacl import validate
from py2neo import Graph as Neo4jGraph, Node, Relationship

### Definindo Variáveis e constantes

In [2]:
IFC_FILE = "../data/Building-Architecture.ifc"
model = ifcopenshell.open(IFC_FILE)
produtos = model.by_type("IfcProduct")

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "ifc123456"
NEO4J_DATABASE = "ifctest"

IFC_NS = Namespace("http://example.org/ifc/")
PROP_NS = Namespace("http://example.org/ifc/property#")
SHACL_NS = Namespace("http://www.w3.org/ns/shacl#")

### Inicializando o grafo

In [ ]:
rdf_graph = Graph()
rdf_graph.bind("ifc", IFC_NS)
rdf_graph.bind("prop", PROP_NS)

### Carregando os dados do modelo IFC no neo4j

os dados do modelo IFC serão carregados no banco de dados Neo4j, onde cada elemento do modelo será representado como um nó no grafo. As relações entre os elementos serão representadas como arestas.